# Agenda

1. What is Polars? What is its relationship with Pandas and the rest of the data ecosystem in Python?
2. Series
3. Data frames
4. Reading in CSV files
5. dtypes
6. Expressions
7. Selecting rows with `df.select`
8. Selecting columns
9. `df.with_columns`
10. `df.filter`
11. Sorting
12. Grouping
13. Optimizing queries and "lazy frames"

# What is Polars?

Data frames are everywhere, but they aren't native to very many languages. We're seeing a growing number of libraries that provide data frames outside of the core of languages. The bad news is that you have to install / choose which data-frame package you want to use in Python. The good news is that we have a few great options to choose from.

Pandas is the 900-pound gorilla in this space.  